# Connect Colab to a Chameleon server

This notebook describes how to connect Colab to a server running on Chameleon. This allows you to run experiments requiring bare metal access, storage, memory, GPU and compute that exceeds the abilities of Colab's hosted runtime, but with Colab's familiar interface (and notebooks stored in your Google Drive). It also allows you to easily go back and forth between the convenience of Colab's hosted runtime and Chameleon's greater capabilities, depending on the needs of your experiment.

## Provision the resource


### Chameleon configuration

You can change your Chameleon project name (if not using the one that is automatically configured in the JupyterHub environment) and the site on which to reserve resources (depending on availability) in the following cell.

In [ ]:
import chi, os

PROJECT_NAME = os.getenv('OS_PROJECT_NAME')
chi.use_site("CHI@UC")
chi.set("project_name", PROJECT_NAME)


You can also change the details of the Chameleon server, e.g. use a different OS image or a different node type depending on availability.

In [ ]:
chi.set("image", "CC-CentOS8-stream-CUDA11")
NODE_TYPE = "gpu_rtx_6000"

### Reservation

The following cell will create a reservation that begins now, and ends in 3 days. You can modify the start and end date as needed.

In [ ]:
from chi import lease


res = []
lease.add_node_reservation(res, node_type=NODE_TYPE, count=1)
lease.add_fip_reservation(res, count=1)
start_date, end_date = lease.lease_duration(days=3)

l = lease.create_lease(f"{os.getenv('USER')}-{NODE_TYPE}", res, start_date=start_date, end_date=end_date)
l = lease.wait_for_active(l["id"])

### Provisioning resources

This cell provisions resources. It will take approximately 10 minutes. You can check on its status in the Chameleon web-based UI.

In [ ]:
from chi import server

reservation_id = lease.get_node_reservation(l["id"])
server.create_server(
    f"{os.getenv('USER')}-{NODE_TYPE}", 
    reservation_id=reservation_id,
    image_name=chi.get("image")
)

Associate an IP address with this server:

In [ ]:
reserved_fip = lease.get_reserved_floating_ips(l["id"])[0]
server.associate_floating_ip(l["id"],reserved_fip)

and wait for it to come up:

In [ ]:
server.wait_for_tcp(reserved_fip, port=22)

## Start a Jupyter server on the server


The following cells will install some basic packages in order to connect your Colab frontend to your Chameleon server. However, you may want to log in to your Chameleon server in order to access its terminal and install or configure packages outside of Colab.

To log in to the resource, use File > New > Terminal in the Chameleon JupyterHub environment, or your local terminal, and run:


In [ ]:
print("cc@" + reserved_fip)

Meanwhile, install Python and JupyterHub on your resource:

In [ ]:
from chi import ssh

node = ssh.Remote(reserved_fip)

In [ ]:
node.run('sudo dnf makecache')
node.run('sudo dnf module -y install python38')
node.run('sudo dnf -y install python38-devel')
node.run('sudo alternatives --set python3 /usr/bin/python3.8')
node.run('sudo alternatives --set python /usr/bin/python3.8')


Install `jupyter_http_over_ws`, which is required in order to connect Colab to this Jupyter instance:

In [ ]:
node.run('python3.8 -m pip install --user  jupyter-core jupyter-client jupyter_http_over_ws traitlets -U --force-reinstall')

And, active `jupyter_http_over_ws`:

In [ ]:
node.run('jupyter serverextension enable --py jupyter_http_over_ws')

The following cell will run a command that does not terminate: 

In [ ]:
node.run("jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0")

In the output of the cell above, look for a URL in this format:
    
```
http://localhost:8888/?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
```

Copy this URL - you will need it in the next step.

## Connect Colab to the server

In a local terminal on your own laptop, run

In [ ]:
print('ssh -L 127.0.0.1:8888:127.0.0.1:8888 cc@' + reserved_fip) 

to set up a tunnel to the Jupyter server. Leave this SSH session open.

Now, you can open Colab in a browser. Click on the drop-down menu for "Connect" in the top right and select "Connect to a local runtime". Paste the URL you copied earlier into the space and click "Connect". Your notebook should now be running on your Colab host (you can put `!hostname` in a cell and run it to verify!)


## Release resources

If you finish with your experimentation before your lease expires,release your resources and tear down your environment by running the following (commented out to prevent accidental deletions):

In [ ]:
# chi.lease.destroy_lease(l["id"])